In [1]:
import pyspark
from pyspark.sql.functions import udf, date_format, to_timestamp, col, desc
from pyspark.sql.types import BooleanType, IntegerType, DoubleType
from utils import datautils
from utils import geoutils

In [2]:
spark = pyspark.sql.SparkSession.builder\
.appName("Rides Preprocessor")\
.master("local")\
.config("spark.local.dir", "/home/atkm/nycTaxi/tmp")\
.getOrCreate()

# Or...
#sparkConf = pyspark.SparkConf()
#sparkConf.set('spark.local.dir', '/home/atkm/nycTaxi/tmp')
#sparkConf.setAppName("Rides Preprocessor")
#sc = pyspark.SparkContext(conf=sparkConf)
#sc.master, sc.getConf().getAll()

In [14]:
def read_csv(path):
    return spark.read.format("csv")\
      .option("header", "true")\
      .option("inferSchema", "true")\
      .load(path)

def drop_minutes(df, colName):
    fmt = "yyyy-MM-dd HH:00:00"
    return df.withColumn(colName, 
                            to_timestamp(
                                date_format(col(colName), fmt), 
                                "yyyy-MM-dd HH:mm:ss"))

def filter_numerical(df, colName):
    df = df.filter(col(colName).rlike(r'\d+(\.\d+)?'))
    return df.withColumn(colName, col(colName).cast(DoubleType()))

# extract weekday and hour from a datetime column
def extract_datetime(df, colName):
    return df.withColumn("weekday", date_format(col(colName),'u'))\
                .withColumn("hour", date_format(col(colName), "H"))\
                .drop(colName)



In [4]:
# Metar pipeline

def load_metar(metarPath):
    metar = read_csv(metarPath)
    metar = metar.select("valid","tmpf", " p01i") # note whitespace in p01i
    return metar.withColumnRenamed('valid', 'datetime')\
        .withColumnRenamed('tmpf', 'fahrenheit')\
        .withColumnRenamed(' p01i', 'precip_in')


def clean_metar(metar):
    precip = metar.select('datetime', 'precip_in')
    precip = clean_precip(precip)
    temperature = metar.select('fahrenheit', 'datetime')
    temperature = clean_temperature(temperature)
    return temperature.join(precip, 'datetime')

def clean_precip(precip):
    if dict(precip.dtypes)['precip_in'] == 'string':
        precip = filter_numerical(precip, 'precip_in')
    precip = precip.filter(date_format(col('datetime'), "m") == 51)
    return drop_minutes(precip, 'datetime')    

def clean_temperature(temperature):
    if dict(temperature.dtypes)['fahrenheit'] == 'string':
        temperature = filter_numerical(temperature, 'fahrenheit')
    temperature = drop_minutes(temperature, 'datetime')
    temperature = temperature.groupby('datetime').mean()
    return temperature.withColumnRenamed('avg(fahrenheit)', 'fahrenheit')

In [5]:
# Rides pipeline
# load -> clean -> add grid columns -> drop minutes 
# -> get_counts -> join weather -> extract hour and weekday.

# TODO: implement datautils.read_rides
def load_rides(ridesPath):
    rides = read_csv(ridesPath)
    # 2014
    colNames = map(lambda name: name.strip(), rides.columns)
    rides = rides.toDF(*colNames)
    return rides.select("pickup_datetime","pickup_latitude", "pickup_longitude")

is_in_nyc = udf(geoutils.is_in_nyc, BooleanType())
get_grid_x = udf(geoutils._get_grid_cell_x, IntegerType())
get_grid_y = udf(geoutils._get_grid_cell_y, IntegerType())

def clean_rides(rides):
    return rides.filter(is_in_nyc(rides.pickup_latitude, 
                           rides.pickup_longitude) == True)

def add_grid_cols(rides):
    return rides.withColumn("grid_x", 
                 get_grid_x(rides.pickup_longitude))\
                .withColumn("grid_y", get_grid_y(rides.pickup_latitude))\
                .drop('pickup_latitude')\
                .drop('pickup_longitude')
        
def get_counts(rides):
    return rides.groupby('pickup_datetime','grid_x','grid_y').count()

# takes an output of load_rides, and prepares it for a join with metar data.
def count_rides(rides):
    rides = clean_rides(rides)
    rides = add_grid_cols(rides)
    rides = drop_minutes(rides, 'pickup_datetime')
    return get_counts(rides)

# takes outputs of count_rides and 
def join_rides_metar(rides, metar):
    return rides.join(metar, rides.pickup_datetime == metar.datetime).drop('pickup_datetime')

## Develop join_rides_metar

In [6]:
rides = load_rides('data/yellow_tripdata_2014-10_tiny.csv')
print(rides.count())
rides = count_rides(rides)
print(rides.count())
rides.show(5)

10000
7812
+-------------------+------+------+-----+
|    pickup_datetime|grid_x|grid_y|count|
+-------------------+------+------+-----+
|2014-10-09 19:00:00|     4|    29|    1|
|2014-10-22 07:00:00|     8|    25|    1|
|2014-10-20 07:00:00|     8|    24|    1|
|2014-10-10 15:00:00|     5|    26|    3|
|2014-10-27 13:00:00|     6|    30|    1|
+-------------------+------+------+-----+
only showing top 5 rows



In [7]:
metar = load_metar('data/metar_data/lga_2014-10.csv')
print(metar.count())
precip = metar.select('datetime','precip_in')
temperature = metar.select('fahrenheit', 'datetime')
metar = clean_metar(metar)
print(metar.count())

859
742


In [8]:
metar.show(5)

+-------------------+------------------+---------+
|           datetime|        fahrenheit|precip_in|
+-------------------+------------------+---------+
|2014-10-20 19:00:00|             57.92|      0.0|
|2014-10-02 05:00:00|             60.98|      0.0|
|2014-10-22 05:00:00|55.507999999999996|     0.13|
|2014-10-24 03:00:00|             51.89|      0.0|
|2014-10-01 03:00:00|             62.06|      0.0|
+-------------------+------------------+---------+
only showing top 5 rows



In [15]:
joined = join_rides_metar(rides, metar)
joined = extract_datetime(joined, 'datetime')
print(joined.count())
joined.show()

7790
+------+------+-----+----------+---------+-------+----+
|grid_x|grid_y|count|fahrenheit|precip_in|weekday|hour|
+------+------+-----+----------+---------+-------+----+
|     8|    21|    1|     57.92|      0.0|      1|  19|
|     7|    21|    1|     57.92|      0.0|      1|  19|
|     5|    23|    1|     57.92|      0.0|      1|  19|
|    22|    39|    1|     57.92|      0.0|      1|  19|
|     3|    29|    2|     57.92|      0.0|      1|  19|
|     6|    27|    1|     57.92|      0.0|      1|  19|
|     5|    28|    1|     57.92|      0.0|      1|  19|
|     5|    26|    2|     57.92|      0.0|      1|  19|
|     4|    26|    1|     57.92|      0.0|      1|  19|
|     9|    24|    1|     57.92|      0.0|      1|  19|
|     3|    32|    1|     57.92|      0.0|      1|  19|
|     4|    27|    1|     57.92|      0.0|      1|  19|
|     5|    25|    2|     57.92|      0.0|      1|  19|
|     5|    24|    2|     57.92|      0.0|      1|  19|
|     5|    27|    1|     57.92|      0.0| 